In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge

%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/katarina74/ml_lessons/main/lesson_2/data/techparams_train.csv")
df.head(2)

,index,back-suspension,battery-capacity,charge-time,compression,consumption-mixed,cylinders-order,cylinders-value,engine-feeding,engine-start,...,configurations_front-brake,configurations_safety-rating,configurations_seats,configurations_tank-volume,supergen_year-stop,models_country-from,models_group,models_light-and-commercial,models_male,target
0,0,9,-1.0,36457,9.0,4.3,0,3,4,2006,...,1,2,13,40.0,2018.0,16,3,0,1,2360
1,2,3,-1.0,44872,8.0,-1.0,3,7,4,1982,...,4,2,13,108.0,1993.0,34,3,0,1,3060


In [3]:
#уберем столбец с индексом
df = df.drop(columns='index')

In [4]:
df.head(2)

,back-suspension,battery-capacity,charge-time,compression,consumption-mixed,cylinders-order,cylinders-value,engine-feeding,engine-start,engine-stop,...,configurations_front-brake,configurations_safety-rating,configurations_seats,configurations_tank-volume,supergen_year-stop,models_country-from,models_group,models_light-and-commercial,models_male,target
0,9,-1.0,36457,9.0,4.3,0,3,4,2006,2018,...,1,2,13,40.0,2018.0,16,3,0,1,2360
1,3,-1.0,44872,8.0,-1.0,3,7,4,1982,1993,...,4,2,13,108.0,1993.0,34,3,0,1,3060


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">

Для поиска лучшего MSE были использованы разные подходы:
1. Преобразование данных (sin, cos, log, sqrt)
2. Использование полиномиальных признаков
3. Использование RobustScaler вместо StandartScaler
4. Использование L2-регуляризации

In [6]:
#стандарт, который пытаемся обойти
X = df.drop(["target"], axis=1)
y = df["target"] 

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

reg = LinearRegression().fit(train_X, train_y)

print("Train R²:", reg.score(train_X, train_y))
print("Test R²:", reg.score(test_X, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X)))

Train R²: 0.45548738550350776
Test R²: 0.46471537322321455
Train MSE: 29843.042074394016
Test MSE: 31015.7531091195


In [7]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#используем синус
train_X_sin = np.sin(train_X)
test_X_sin = np.sin(test_X)

#Стандартизация после преобразования исходных данных
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X_sin)
test_X_scaled = scaler.transform(test_X_sin)

reg = LinearRegression().fit(train_X_scaled, train_y)

print("Train R²:", reg.score(train_X_scaled, train_y))
print("Test R²:", reg.score(test_X_scaled, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_scaled)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_scaled)))

Train R²: 0.20457402504499456
Test R²: 0.2173806401722873
Train MSE: 43594.82260957065
Test MSE: 45346.956793802325


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование в качестве преобразования данных синус только ухудшил ситуацию, проверяем дальше.

In [9]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#используем косинус
train_X_cos = np.cos(train_X)
test_X_cos = np.cos(test_X)

#Стандартизация после преобразования исходных данных
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X_cos)
test_X_scaled = scaler.transform(test_X_cos)

reg = LinearRegression().fit(train_X_scaled, train_y)

print("Train R²:", reg.score(train_X_scaled, train_y))
print("Test R²:", reg.score(test_X_scaled, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_scaled)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_scaled)))

Train R²: 0.27985343800828266
Test R²: 0.27838215092880414
Train MSE: 39468.99222733703
Test MSE: 41812.37406479675


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование косинуса улучшило ситуацию, однако это тоже плохой результат

In [11]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#используем синус
train_X_sin = np.sin(train_X)
test_X_sin = np.sin(test_X)

#воспользуемся полиномиальными признаками
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
train_X_poly = poly.fit_transform(train_X_sin)
test_X_poly = poly.transform(test_X_sin)

reg = LinearRegression().fit(train_X_poly, train_y)

print("Train R²:", reg.score(train_X_poly, train_y))
print("Test R²:", reg.score(test_X_poly, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_poly)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_poly)))

Train R²: 0.3372256097029328
Test R²: 0.3309846149418668
Train MSE: 36324.60201818453
Test MSE: 38764.453472373614


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование синуса и полиномиального признака еще улучшают ситуацию, однако это не превосходит стандартный результат

In [13]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#используем косинус
train_X_cos = np.cos(train_X)
test_X_cos = np.cos(test_X)

#воспользуемся полиномиальными признаками
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
train_X_poly = poly.fit_transform(train_X_cos)
test_X_poly = poly.transform(test_X_cos)

reg = LinearRegression().fit(train_X_poly, train_y)

print("Train R²:", reg.score(train_X_poly, train_y))
print("Test R²:", reg.score(test_X_poly, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_poly)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_poly)))

Train R²: 0.39856883371522
Test R²: 0.3918807179676126
Train MSE: 32962.57078797975
Test MSE: 35235.97833545988


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование косинуса и полиномиального признака выглядит лучше на фоне синуса, но является далеким от результата

In [15]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#используем логарифм, +1, чтобы избежать бесконечности и проблем
train_X_log = np.log1p(train_X + 1)
test_X_log = np.log1p(test_X + 1)

#Стандартизация после преобразования исходных данных
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X_log)
test_X_scaled = scaler.transform(test_X_log)

reg = LinearRegression().fit(train_X_scaled, train_y)

print("Train R²:", reg.score(train_X_scaled, train_y))
print("Test R²:", reg.score(test_X_scaled, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_scaled)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_scaled)))

Train R²: 0.42508702590571945
Test R²: 0.4358337156317418
Train MSE: 31509.191189033765
Test MSE: 32689.229828660697


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование логарифма приближается к стандарту, но не достигает его, нужно искать новые способы

In [17]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#Стандартизация после преобразования исходных данных
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

#воспользуемся L2-регуляризацией
reg = Ridge(alpha=1.0).fit(train_X_scaled, train_y)

print("Train R²:", reg.score(train_X_scaled, train_y))
print("Test R²:", reg.score(test_X_scaled, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_scaled)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_scaled)))

Train R²: 0.4554873784882104
Test R²: 0.464715056883237
Train MSE: 29843.042458880638
Test MSE: 31015.77143866261


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование L2-регуляризации дает результаты чуть хуже, чем имеющийся стандарт, который мы пытаемся обойти

In [19]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#Стандартизация после преобразования исходных данных
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

reg = LinearRegression().fit(train_X_scaled, train_y)

print("Train R²:", reg.score(train_X_scaled, train_y))
print("Test R²:", reg.score(test_X_scaled, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_scaled)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_scaled)))

Train R²: 0.45548738550350765
Test R²: 0.4647153732232171
Train MSE: 29843.042074394023
Test MSE: 31015.75310911936


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Используя стандартизацию, мы получаем примерно тот же результат, который мы пытаемся улучшить

In [21]:
X = df.drop(["target"], axis=1)
y = df["target"] 

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#исключение выбросов
scaler = RobustScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

reg = LinearRegression().fit(train_X_scaled, train_y)

print("Train R²:", reg.score(train_X_scaled, train_y))
print("Test R²:", reg.score(test_X_scaled, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_scaled)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_scaled)))

Train R²: 0.45548738550350765
Test R²: 0.4647153732232171
Train MSE: 29843.042074394023
Test MSE: 31015.75310911936


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование RobustScaler дает тот же результат, что и стандартизация, пробуем дальше

In [23]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#извлечем корень, перед этим возведя все данные в квадрат, тем самым избавившись от отрицательных значений
train_X_sqrt = np.sqrt(train_X ** 2)
test_X_sqrt = np.sqrt(test_X ** 2)

reg = LinearRegression().fit(train_X_sqrt, train_y)

print("Train R²:", reg.score(train_X_sqrt, train_y))
print("Test R²:", reg.score(test_X_sqrt, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_sqrt)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_sqrt)))

Train R²: 0.45676229336648566
Test R²: 0.46619494988346866
Train MSE: 29773.168341475986
Test MSE: 30930.02267318902


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Данный способ дает лучший результат, проверим другие способы

In [25]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#воспользуемся полиномиальными признаками
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
train_X_poly = poly.fit_transform(train_X)
test_X_poly = poly.transform(test_X)

reg = LinearRegression().fit(train_X_poly, train_y)

print("Train R²:", reg.score(train_X_poly, train_y))
print("Test R²:", reg.score(test_X_poly, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_poly)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_poly)))

Train R²: 0.6115380894252873
Test R²: 0.6147896092347913
Train MSE: 21290.388565746107
Test MSE: 22320.070066244072


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
С использованием полиномиального признака получен достаточно хороший результат, проверим, можно ли еще улучшить

In [27]:
X = df.drop(["target"], axis=1)
y = df["target"]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

#извлечем корень, перед этим возведя все данные в квадрат, тем самым избавившись от отрицательных значений
train_X_sqrt = np.sqrt(train_X ** 2)
test_X_sqrt = np.sqrt(test_X ** 2)

#воспользуемся полиномиальными признаками
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
train_X_poly = poly.fit_transform(train_X_sqrt)
test_X_poly = poly.transform(test_X_sqrt)

reg = LinearRegression().fit(train_X_poly, train_y)

print("Train R²:", reg.score(train_X_poly, train_y))
print("Test R²:", reg.score(test_X_poly, test_y))
print("Train MSE:", mean_squared_error(train_y, reg.predict(train_X_poly)))
print("Test MSE:", mean_squared_error(test_y, reg.predict(test_X_poly)))

Train R²: 0.6130641740540038
Test R²: 0.616274662276814
Train MSE: 21206.74861586912
Test MSE: 22234.022314821297


<div style="background-color:#CAEDFB; padding: 10px; border: 1px solid #aaa;">
Использование корня и полиномиального признака помогло нам достигнуть наилучшего результата